# Bike Share Rebalancing With Mathematical Optimization

Bike share systems have become an effective commuting method globally for everyday urban dwellers as well as tourists.

Citi-Bike in NYC being the largest Bike-Sharing network had 1,588 active stations and 25,575 active bikes in July 2022.

Over 3 million rides were completed July 2022 that cover NYC/Hoboken/Jersey City, with around 150,000 active annual members.

During rush hours there are many bike stations that have a high demand for bikes, which means their out-flow of bikes is greater than their in-flow in these stations. 

Meanwhile there are stations that have a high demand for docks (riders return their bikes to these stations) which means their in-flow of bikes is greater than their out-flow.

Lack of available bikes or docks in high-demand stations can cause major imbalance in the bike sharing network and result in customer dissatisfaction and lost revenue.

To tackle this problem, bikes are relocated between stations to create a balance between supply and demand.

## Problem Statement and Solution Approach

Using historical Citi-bike data in NYC and Jersey area during July 2022, we like to know:
- What is the demand for bikes per hour at each station during the first week of August?
- Knowing the demand, how can we minimize loss of sale?

Loss of sale is caused by lack of bikes when customers demand them. So, bikes should be transfered from stations with higher in-flow of bikes to those with higher out-flow of bikes. 

So, first, number of bikes to be added to or removed from each station during each hour should be determined. Then, the physical transfer of bikes between stations should be scheduled. 

In this notebook, we'll focus on the first part and at the end, discuss how the second part can be solved.
We'll use a mixture of Machine Learning (ML) and Mathematical Optimization (MO) to solve this problem. 

**Solution Approach**
The solution approach is comprised of two steps:
- **Step 1**: We use the historical Citi-bike data in NYC and Jersey area during July 2022 and use an ML model to predict the number of in-flow and out-flow of bikes per hour at each station for the first week of August. This is done in [predict_bike_flow](predict_bike_flow.ipynb) Notebook.
- **Step 2**: We use an MO model to decide how many bikes should be added to or removed from each station during each hour so that the total loss of sale is minimized.

To ensure that everyone can run the notebook with the gurobi restricted license, we reduce the size of the data. To achieve that, we focus on the top 100 stations during the morning rush hours (7 am to 9 am).

The top stations are chosen using the PageRank algorithm.

# Install Required Packages

In [ ]:
%pip install gurobipy
%pip install pandas

# For Google Colab Only
If you like to run the notebook in Google Colab, follow these steps:
- Click on [this link](https://colab.research.google.com/github/decision-spot/bike_share/blob/main/bike_rebalancing.ipynb)
This should open up the notebook in Google Colab.
- To get all the files, run the following cells to clone to repo and change the current working directory path.

In [ ]:
!git clone https://github.com/decision-spot/bike_share.git

In [ ]:
import os
from google.colab import files
os.chdir('bike_share')

# Import Packages

In [1]:
import datetime
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# Optimization Problem

## Problem Definition

We want to minimize the total loss of sale. Loss of sale at each station and in each hour can be defined as the difference between the total demand of bikes (number of bikes that start their trip from the station) and total supply of bikes.

Total supply is comprised of number of bikes that end their trip at the station plus all the existing bikes at the station (a.k.a inventory) plus number of bikes that are added or removed from that station in that hour through some bike transfers. 

**Assumptions:**
- Inventory at the begininng of first hour (in our case, hour 7) is zero.
- At any given hour, we have access to a limited number of bikes that can be added to the stations in hope of helping reduce the imbalance without yet transfering the bikes between stations (since this analysis is during morning rush hours).

## Load Required Data

In [2]:
stations = pd.read_csv('top_stations.csv', index_col='station')
station_names = list(stations.index)
stations.head()

,capacity,lat,lon,region
station,,,,
Cleveland Pl & Spring St,33,40.722104,-73.997249,71
Lafayette St & Jersey St,40,40.724561,-73.995653,71
Grand Army Plaza & Central Park S,96,40.764397,-73.973715,71
Broadway & E 14 St,114,40.734546,-73.990741,71
Lafayette St & E 8 St,47,40.730207,-73.991026,71


In [3]:
stations_flow = pd.read_csv('stations_flow.csv')
stations_flow['datetime'] = pd.to_datetime(stations_flow['datetime'])
stations_flow.head()

,station,datetime,start_forecast,end_forecast
0,1 Ave & E 62 St,2022-08-01 00:00:00,3.363690,5.054894
1,1 Ave & E 62 St,2022-08-01 01:00:00,2.006569,3.079629
2,1 Ave & E 62 St,2022-08-01 02:00:00,3.413499,1.313538
3,1 Ave & E 62 St,2022-08-01 03:00:00,2.323314,1.352228
4,1 Ave & E 62 St,2022-08-01 04:00:00,1.591569,1.504905


The `stations_flow` data contains the prediction for the first 5 days of August 2022 and during all the hours. Our analysis is for morning rush hours, between 7 to 9 am. Also, we can run our MO model daily. For now, we only focus on the first day but at the end, we will show the full model and how it can be run daily.

In [4]:
# Pandas will give a few SettingWithCopyWarning here when new columns are created. 
# They are false alarms. So, we suppress them.
pd.options.mode.chained_assignment = None
morning_flow = stations_flow[stations_flow['datetime'].dt.hour.between(7, 9)]
morning_flow['date'] = morning_flow['datetime'].dt.date
morning_flow['time'] = morning_flow['datetime'].dt.hour
morning_flow = morning_flow.round(0)  # to have integer values for forecast of number of bikes
# For now, let's run the MO model for the first date: 08/01/2022
flow_df = morning_flow.loc[morning_flow['date'] == datetime.date(2022, 8, 1)]
flow_df.set_index(['station', 'time'], inplace=True)

## Problem Formulation

Let's define our notations for the MO model. We want to run the model for every hour and every station. So, let's define the following two sets:

**Sets**
- $I\quad$: Set of stations
- $T\quad$: Set of hours 

We also have the following information from `stations` and `flow_df` dataframes:

**Parameters**
- $e_{i,t}\quad$: number of bikes that end their trip at station $i$ at hour $t$ (a.k.a. supply)
- $s_{i,t}\quad$: number of bikes that start their trip at station $i$ at hour $t$ (a.k.a. demand)
- $c_{i}\quad$: capacity of station $i$

We know it's not easy to transfer bikes between stations during rush hours and heavy traffic. To mitigate loss due to unavailable bikes at high-demand stations, we assume there is a small reserve of bikes available at the beginning of an hour that we can allocate to the stations. We show that with $N$: 
- $N\quad$: Number of bikes at hand that we can assign to stations at a given hour

In [5]:
num_bikes = 100  # N: Number of bikes at hand that we can assign to stations at a given hour
time_rng = morning_flow['time'].drop_duplicates().values  # set T

In [10]:
station_time = flow_df.index  # pair of (i,t) index
start_forecast = flow_df.start_forecast  # s
end_forecast = flow_df.end_forecast  # e
capacity = stations.capacity  # c

## Decision Variables

First of all, we like to find out how many bikes should be added to or removed from each station during each hour. We show that with $b_{i,t}$:
- $b_{i,t}\quad$: number of bikes to be added to or removed from station $i$ at hour $t$

Variable $b_{i,t}$ can be non-negative (for adding bikes) and negative (for removing bikes).

Another variable is the inventory of bikes at each station and at the beginning of each hour. Since the inventory depends on the value of $b_{i,t}$, it's another decision variable. We show that with $q_{i,t}$:
- $q_{i,t}\quad$: inventory of bikes at station $i$ at the beginning of hour $t$

For simplicity, we assume that there is no inventory at the beginning of the first hour. In our case, this means no initial inventory at 7 am.

The goal of the model is to reduce the lost sale at each station per hour. This value, also depends on the value of decision variable $b_{i,t}$ and as a result, is a decision variable itself. We show that with $l_{i,t}$:
- $l_{i,t}\quad$: lost sale at station $i$ at hour $t$

With these initial decision variables, we can start the MO model:

In [11]:
mdl = gp.Model('bike_rebalancing')
# Variables

b = mdl.addVars(station_names, time_rng, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name='b')  # we'll set a better lb later
# b = mdl.addVars(station_time, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name='b')  # alternatively
q = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='q')
l = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='l')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-13


## Constraints

First, we set up lowerbound and upperbound values for the number of bikes that can be added to or removed from a station in each hour.

In each hour, we have $s_{i,t}$ bikes that start their trip from the station and $e_{i,t}$ bikes that end their trip at the station. If $s_{i,t} \ge e_{i,t}$, demand is exceeding supply. In this case, we may choose to add some bikes to this station to reduce the loss (we can also add nothing). One thing to remember is that we cannot add more bikes than the station's capacity.

On the other hand, if $s_{i,t} \le e_{i,t}$, we may choose to remove some of the excess bikes from that station (we can also remove nothing).

The above descriptions, help us to define the bounds on $b_{i,t}$.
For lowerbound we have:

\begin{align}
&b_{i,t} \ge \min(0, s_{i,t} - e_{i,t}) &\quad \forall i \in I, t \in T \tag{1}\\
\end{align}

And for upperbound we have:

\begin{align}
&b_{i,t} \le \min(c_i, c_i + s_{i,t} - e_{i,t}) &\quad \forall i \in I, t \in T \tag{2}\\
\end{align}


In [17]:
# LB of b
mdl.addConstrs(
    (b[i, t] >= min(0,
                    start_forecast[i, t] - end_forecast[i, t])
#      for i, t in station_time), 'lb_b')
    for i in station_names for t in time_rng), 'lb_b')

# UB of b
mdl.addConstrs(
    (b[i, t] <= min(capacity[i], 
                    capacity[i] + start_forecast[i, t] - end_forecast[i, t])
     for i, t in station_time), 'ub_b');  # add ";" here to stop the cell to print the constraints

Next, we set up the initial inventory (i.e. inventory at hour $t_0$) to be 0. 

\begin{align}
&q_{i,t_0} = 0 &\quad \forall i \in I \tag{3}\\
\end{align}

In [18]:
t0 = 7
# setting up initial inventory
mdl.addConstrs((q[i, t0] == 0 for i in stations.index), name='initial_inv');

Our next constraint is the definition of the inventory at a station at the end of an hour.
The inventory is defined as the difference between all the bikes that are added to a station minus all the bikes that are removed. Since it's possible that demand of bikes at a station exceeds the total supply of bikes (which will cause loss of sale), we need to ensure that inventory at a station is a non-negative number.

\begin{align}
&q_{i,t} = \max(0, e_{i,t-1} - s_{i,t-1} + b_{i,t-1} + q_{i,t-1}) &\quad \forall i \in I, t \in T/t_0 \tag{4}\\
\end{align}

To write this constraint in Gurobi, we can take advantage of Gurobi's [`max_()`](https://www.gurobi.com/documentation/9.5/refman/py_max_.html) function. This function accepts a list of decision variables, and if desired, a constant. Our constant is 0. But $e_{i,t-1} - s_{i,t-1} + b_{i,t-1} + q_{i,t-1}$ is a linear expression and not a decision variable. However, the fix is very simple. We can define an auxiliary variable that is equal to the linear expression and then use that auxiliary variable in the inventory's defintion. In other words, we first create $a_{i,t}$ as a new decision variable:

- $a_{i,t}\quad$: auxiliary variable

Then, we define $a_{i,t}$ as:

\begin{align}
&a_{i,t} = e_{i,t-1} - s_{i,t-1} + b_{i,t-1} + q_{i,t-1} &\quad \forall i \in I, t \in T/t_0 \tag{5}\\
\end{align}

Equation 4 can then be simplified to:

\begin{align}
&q_{i,t} = \max(0, a_{i,t}) &\quad \forall i \in I, t \in T/t_0 \tag{6}\\
\end{align}

We replace equation 4 with equations 5 and 6.

In [20]:
a = mdl.addVars(station_time, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name='a')  # auxiliary variable
# defintion of auxiliary variable
mdl.addConstrs((a[i, t] == end_forecast[i, t - 1]
                - start_forecast[i, t - 1] + b[i, t - 1] + q[i, t - 1]
                for i, t in station_time if t != t0), name='aux_def')
# definition of inventory
mdl.addConstrs((q[i, t] == gp.max_(a[i, t], 0) for i, t in station_time), name='inv_def');

The next constraint is an upperbound on the inventory at each station. We need to ensure the inventory does not exceed the capacity of the station.

\begin{align}
&q_{i,t} \le c_i &\quad \forall i \in I, t \in T \tag{7}\\
\end{align}

In [21]:
# UB of inventory
mdl.addConstrs((q[i, t] <= capacity[i] for i, t in station_time), 'ub_inv');

Next, we need to define how the loss of sale is calculated. Loss of sale is the difference between the demand and all the supply of bikes at a station. Of course, if the supply is greater than the demand, there is no loss. So, we need to ensure that loss only considers non-negative values. This can be achived by:

\begin{align}
&l_{i,t} = \max(0, s_{i,t} - e_{i,t} - b_{i,t} - q_{i,t}) &\quad \forall i \in I, t \in T \tag{8}\\
\end{align}

In [22]:
# loss definition
mdl.addConstrs(
    (l[i, t] >= start_forecast[i, t]
     - end_forecast[i, t] - b[i, t] - q[i, t]
     for i, t in station_time), 'loss_def');

We assumed that we have a small reserve of bikes at the beginning of each hour to allocate to stations. This limit is on the total number of bikes added to the stations. Namely, it's a limit on the positive values of $b_{i,t}$. There are several ways to formulate this constraint. We go with the most intuitive one, namely defining a new variable $y_{i,t}$ to be equal to the non-negative values of $b_{i,t}$. In other words:

- $y_{i,t}\quad$: number of bikes added to station $i$ at hour $t$

where:
\begin{align}
&y_{i,t} = \max(0, b_{i,t}) &\quad \forall i \in I, t \in T \tag{9}\\
\end{align}

In [23]:
y = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='y')  # number of bikes added to station
# definition of y
mdl.addConstrs((y[i, t] == gp.max_(b[i, t], 0) for i, t in station_time), name='y_def');

We can now write the constraint, limiting total number of bikes added to the stations.

\begin{align}
&\sum_{i} y_{i,t} \le N &\quad \forall t \in T \tag{10}\\
\end{align}

In [24]:
# limit on number of bikes added
mdl.addConstrs((y.sum('*', t) <= num_bikes for t in time_rng), name='total_bikes');

## Objective

The objective is to minimize total loss of sale. 

$$\min \sum_{i,t} l_{i,t}$$

In [25]:
mdl.setObjective(l.sum(), GRB.MINIMIZE)

We can now tell Gurobi that the model is complete and it can solve the problem.

In [26]:
mdl.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2787 rows, 1728 columns and 4416 nonzeros
Model fingerprint: 0xf7662c4a
Model has 864 general constraints
Variable types: 1728 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 2321 rows and 1205 columns
Presolve time: 0.04s
Presolved: 466 rows, 523 columns, 1290 nonzeros
Variable types: 377 continuous, 146 integer (146 binary)
Found heuristic solution: objective 109.0000000

Root relaxation: objective 2.705556e+01, 324 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.05556    0   50  109.00000   27.05556  75.2%     -

## Post Processing

In [27]:
df = pd.DataFrame()
if mdl.status == GRB.Status.OPTIMAL:
    df = flow_df.copy()
    df = df.merge(stations[['capacity']], left_on='station', right_index=True)
    df[['bikes_added_or_removed', 'loss_sale', 'beginning_inventory']] = 0
    for k, v in b.items():
        df.loc[k, 'bikes_added_or_removed'] = v.x
        df.loc[k, 'beginning_inventory'] = q[k].x
        df.loc[k, 'loss_sale'] = l[k].x
    df.reset_index(inplace=True)
    print(f'Total Loss : {mdl.objVal}')
    total_bikes_added = df.groupby('time').apply(
        lambda x: x[x.bikes_added_or_removed >= 0]['bikes_added_or_removed'].sum())
    print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
else:
    print('Could not find a feasible solution!')
df.head(10)

Total Loss : 109.0
Total number of bikes added in each hour:
 time
7    100
8    100
9    100
dtype: int64


,station,time,datetime,start_forecast,end_forecast,date,capacity,bikes_added_or_removed,loss_sale,beginning_inventory
0,1 Ave & E 62 St,7,2022-08-01 07:00:00,11.0,10.0,2022-08-01,45,0,1,0
1,1 Ave & E 62 St,8,2022-08-01 08:00:00,13.0,13.0,2022-08-01,45,0,0,0
2,1 Ave & E 62 St,9,2022-08-01 09:00:00,16.0,19.0,2022-08-01,45,-3,0,0
3,1 Ave & E 68 St,7,2022-08-01 07:00:00,17.0,31.0,2022-08-01,62,-14,0,0
4,1 Ave & E 68 St,8,2022-08-01 08:00:00,21.0,40.0,2022-08-01,62,-19,0,0
5,1 Ave & E 68 St,9,2022-08-01 09:00:00,27.0,47.0,2022-08-01,62,-20,0,0
6,1 Ave & E 78 St,7,2022-08-01 07:00:00,9.0,6.0,2022-08-01,57,13,0,0
7,1 Ave & E 78 St,8,2022-08-01 08:00:00,17.0,4.0,2022-08-01,57,3,0,10
8,1 Ave & E 78 St,9,2022-08-01 09:00:00,17.0,6.0,2022-08-01,57,0,11,0
9,10 Ave & W 14 St,7,2022-08-01 07:00:00,3.0,5.0,2022-08-01,55,-2,0,0


## Putting it all together

**Sets**
- $I\quad$: Set of stations
- $T\quad$: Set of hours 

**Parameters**
- $e_{i,t}\quad$: number of bikes that end their trip at station $i$ at hour $t$ (a.k.a. supply)
- $s_{i,t}\quad$: number of bikes that start their trip at station $i$ at hour $t$ (a.k.a. demand)
- $c_{i}\quad$: capacity of station $i$
- $N\quad$: Number of bikes at hand that we can assign to different stations at a given hour

**Variables**
- $b_{i,t}\quad$: number of bikes to be added to or removed from station $i$ at hour $t$
- $q_{i,t}\quad$: inventory of bikes at station $i$ at the beginning of hour $t$
- $l_{i,t}\quad$: lost sale at station $i$ at hour $t$
- $y_{i,t}\quad$: number of bikes added to station $i$ at hour $t$
- $a_{i,t}\quad$: auxiliary variable needed to define inventory's relationship with other variables. We use this in the code

\begin{align}
&\min \sum_{i,t} l_{i,t}&\\
\mbox{s.t: }\\
&b_{i,t} \ge \min(0, s_{i,t} - e_{i,t}) &\quad \forall i \in I, t \in T \tag{1}\\
&b_{i,t} \le \min(c_i, c_i + s_{i,t} - e_{i,t}) &\quad \forall i \in I, t \in T \tag{2}\\
&q_{i,t_0} = 0 &\quad \forall i \in I \tag{3}\\
&a_{i,t} = e_{i,t-1} - s_{i,t-1} + b_{i,t-1} + q_{i,t-1} &\quad \forall i \in I, t \in T/t_0 \tag{4}\\
&q_{i,t} = \max(0, a_{i,t}) &\quad \forall i \in I, t \in T/t_0 \tag{5}\\
&q_{i,t} \le c_i &\quad \forall i \in I, t \in T \tag{6}\\
&l_{i,t} = \max(0, s_{i,t} - e_{i,t} - b_{i,t} - q_{i,t}) &\quad \forall i \in I, t \in T \tag{7}\\
&y_{i,t} = \max(0, b_{i,t}) &\quad \forall i \in I, t \in T \tag{8}\\
&\sum_{i} y_{i,t} \le N &\quad \forall t \in T \tag{9}\\
&y_{i,t}, q_{i,t}, l_{i,t} \ge0 &\quad \forall i \in I, t \in T \tag{10}\\
&b_{i,t} \quad \mbox{urs} &\quad \forall i \in I, t \in T \tag{11}\\
\end{align}

In [28]:
def bike_rebalancing(flow_df, num_bikes):
    station_time = flow_df.index  # pair of (i,t) index
    start_forecast = flow_df.start_forecast  # s
    end_forecast = flow_df.end_forecast  # e
    capacity = stations.capacity  # c
    mdl = gp.Model('bike_rebalancing')

    # Variables
    b = mdl.addVars(station_time, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name='b')
    q = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='q')
    l = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='l')
    y = mdl.addVars(station_time, lb=0, vtype=GRB.CONTINUOUS, name='y')
    a = mdl.addVars(station_time, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name='a')

    # Constraints
    # constr 1) b_{i,t} >= min(0, s_{i,t} - e_{i,t})    for all i in I, t in T
    mdl.addConstrs(
        (b[i, t] >= min(0, start_forecast[i, t] - end_forecast[i, t])
         for i, t in station_time), 'lb_b')

    # constr 2) b_{i,t} <= min(c_i, c_i + s_{i,t} - e_{i,t})    for all i in I, t in T
    mdl.addConstrs(
        (b[i, t] <= min(capacity[i], 
                        capacity[i] + start_forecast[i, t] - end_forecast[i, t])
         for i, t in station_time), 'ub_b')

    # constr 3) We can assume that initial inventory at the first hour (in our case, t=7) = 0
    # q_{i,t} = 0     for all i, for t=t0 (where t0=7)
    t0 = 7
    mdl.addConstrs((q[i, t0] == 0 for i in stations.index), name='initial_inv')
    # mdl.addConstrs((a[i, t0] == 0 for i in stations.index), name='initial_aux')

    # constr 4)
    # if we want to use gurobi's max_() function, we need to pass variables and not linear expressions.
    # So, we first need to define an auxiliary variable to replace the linear expression
    # and then pass that to definition of inventory
    # a_{i,t} = e_{i,t-1} - s_{i,t-1} + b_{i,t-1} + q_{i,t-1}   for all i in I, t in T/t0
    mdl.addConstrs((a[i, t] == end_forecast[i, t - 1]
                    - start_forecast[i, t - 1] + b[i, t - 1] + q[i, t - 1]
                    for i, t in station_time if t != t0), name='aux_def')

    # constr 5) q_{i,t} = max(0, a_{i,t})   for all i in I, t in T
    mdl.addConstrs((q[i, t] == gp.max_(a[i, t], 0) for i, t in station_time), name='inv_def')

    # constr 6) q_{i,t} <= c_{i}   for all i in I, t in T
    mdl.addConstrs((q[i, t] <= capacity[i] for i, t in station_time), 'ub_inv')

    # constr 7) l_{i,t} = max(0, s_{i,t} - e_{i,t} - b_{i,t} - q_{i,t})    for all i in I, t in T
    mdl.addConstrs(
        (l[i, t] >= start_forecast[i, t]
         - end_forecast[i, t] - b[i, t] - q[i, t]
         for i, t in station_time), 'loss_def')

    # constr 8) y_{i,t} = max(0, b_{i,t})    for all i in I, t in T
    mdl.addConstrs((y[i, t] == gp.max_(b[i, t], 0) for i, t in station_time), name='y_def')

    # constr 9) only bikes that are added to the stations are counted toward the limits
    # sum_i y_{i,t} <= N    for all t in T
    mdl.addConstrs((y.sum('*', t) <= num_bikes for t in time_rng), name='total_bikes')

    # Objectives
    mdl.setObjective(l.sum(), GRB.MINIMIZE)  # Min sum_{i,t} l_{i,t}
    mdl.optimize()

    # create output
    df = pd.DataFrame()
    if mdl.status == GRB.Status.OPTIMAL:
        df = flow_df.copy()
        df = df.merge(stations[['capacity']], left_on='station', right_index=True)
        df[['bikes_added_or_removed', 'loss_sale', 'beginning_inventory']] = 0
        for k, v in b.items():
            df.loc[k, 'bikes_added_or_removed'] = v.x
            df.loc[k, 'beginning_inventory'] = q[k].x
            df.loc[k, 'loss_sale'] = l[k].x
        df.reset_index(inplace=True)
        print(f'Total Loss : {mdl.objVal}')
        total_bikes_added = df.groupby('time').apply(
            lambda x: x[x.bikes_added_or_removed >= 0]['bikes_added_or_removed'].sum())
        print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
    else:
        print('Could not find a feasible solution!')
    return df

In [29]:
# run the model daily
all_outputs = []
g = morning_flow.set_index(['station', 'time']).groupby('date')
for x in g.groups:
    df = g.get_group(x)
    odf = bike_rebalancing(df, num_bikes)
    all_outputs.append(odf)
output_df = pd.concat(all_outputs)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1443 rows, 1440 columns and 2688 nonzeros
Model fingerprint: 0x2a9ff18e
Model has 576 general constraints
Variable types: 1440 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 1150 rows and 1008 columns
Presolve time: 0.01s
Presolved: 293 rows, 432 columns, 934 nonzeros
Variable types: 368 continuous, 64 integer (64 binary)
Found heuristic solution: objective 109.0000000

Root relaxation: objective 2.705556e+01, 265 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.05556    0   28  109.00000   27.05556  75.2%     -   

     0     0   80.22339    0   10   92.00000   80.22339  12.8%     -    0s
     0     0   91.50000    0    4   92.00000   91.50000  0.54%     -    0s
     0     0   91.50000    0    4   92.00000   91.50000  0.54%     -    0s
     0     0     cutoff    0        92.00000   92.00000  0.00%     -    0s

Cutting planes:
  Implied bound: 7
  MIR: 26
  Flow cover: 79
  Relax-and-lift: 2

Explored 1 nodes (712 simplex iterations) in 0.10 seconds (0.02 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 92 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.200000000000e+01, best bound 9.200000000000e+01, gap 0.0000%
Total Loss : 92.0
Total number of bikes added in each hour:
 time
7    100
8    100
9    100
dtype: int64


In [54]:
output_df.head(10)

,station,time,datetime,start_forecast,end_forecast,date,capacity,bikes_added_or_removed,loss_sale,beginning_inventory
0,1 Ave & E 62 St,7,2022-08-01 07:00:00,11.0,10.0,2022-08-01,45,1.0,0.0,0.0
1,1 Ave & E 62 St,8,2022-08-01 08:00:00,13.0,13.0,2022-08-01,45,0.0,0.0,0.0
2,1 Ave & E 62 St,9,2022-08-01 09:00:00,16.0,19.0,2022-08-01,45,-3.0,0.0,0.0
3,1 Ave & E 68 St,7,2022-08-01 07:00:00,17.0,31.0,2022-08-01,62,-14.0,0.0,0.0
4,1 Ave & E 68 St,8,2022-08-01 08:00:00,21.0,40.0,2022-08-01,62,-19.0,0.0,0.0
5,1 Ave & E 68 St,9,2022-08-01 09:00:00,27.0,47.0,2022-08-01,62,-20.0,0.0,0.0
6,1 Ave & E 78 St,7,2022-08-01 07:00:00,9.0,6.0,2022-08-01,57,6.0,0.0,0.0
7,1 Ave & E 78 St,8,2022-08-01 08:00:00,17.0,4.0,2022-08-01,57,10.0,0.0,3.0
8,1 Ave & E 78 St,9,2022-08-01 09:00:00,17.0,6.0,2022-08-01,57,11.0,0.0,0.0
9,10 Ave & W 14 St,7,2022-08-01 07:00:00,3.0,5.0,2022-08-01,55,-2.0,0.0,0.0


# Model Enhancement
Looking at the `output_df`, you may notice stations that see a transfer of bikes (addition or removal) at every hour. Even worse are those stations where some bikes are removed from them in one hour but then bikes are added to them in the next hour. We know these additions/removals consume time and money. What can we do to avoid this situation?

One way to formulate this is to introduce a fixed cost for the use of the truck (or the bicycle trailer) that transfers the bikes and then add this term to the objective function. With a cost associated with the transfer, the model is incentivized to use fewer number of transfers.

So first, we should calculate the number of times a transfer has occured. Any time bikes are added to a station or removed from a station, a transfer has happened. So, we need a way to link addition of bikes and removal of bikes to a transfer. One way to achieve this is to introduce two new binary variables as follows:

- $x_{i,t}\quad$: 1 if any bike is added to station $i$ at hour $t$; 0 otherwise
- $w_{i,t}\quad$: 1 if any bike is removed from station $i$ at hour $t$; 0 otherwise

Next, we need to establish the relationship between $b_{i,t}$ with $x_{i,t}$ and $w_{i,t}$. Basically, we want to say:
if $b_{i,t} \ge 0$, then $x_{i,t} = 1$ and if $b_{i,t} \le 0$, then $w_{i,t} = 1$. 

We introduce the following two constraints:

\begin{align}
&b_{i,t} \le M x_{i,t} &\quad \forall i \in I, t \in T \tag{11}\\
&-b_{i,t} \le M w_{i,t} &\quad \forall i \in I, t \in T \tag{12}\\
\end{align}

where $M$ is a large number.

Constraint 11 ensures that if $b_{i,t}\ge 0$, then $x_{i,t} = 1$. But by itself, this constraint cannot make $x_{i,t} =0$ if $b_{i,t} \le 0$. The same is true with constraint 12. It ensures that negative values of $b_{i,t}$ make $w_{i,t} = 1$. However, it cannot force $w_{i,t} = 0$ if $b_{i,t} \ge 0$.

This can be achieved by the objective function.

Total number of transfers is equal to sum of $x_{i,t}$ and $w_{i,t}$ and our goal is to minimize number of transfers. So, we add these terms to the objective function. In other words, our new objective function is:

$$\min \sum_{i,t} (l_{i,t} + x_{i,t} + w_{i,t})$$

Since minimizing total transfers is desired, the objective function tries to make both $x_{i,t}$ and $w_{i,t}$ to be zero. Along with constraints 11 and 12, this means that for cases where $x_{i,t}$ and $w_{i,t}$ can take either 0 or 1, objective function forces them to get a value of 0. Moreover, since any extra transfer causes either $x_{i,t}$ or $w_{i,t}$ to be 1, the model is incentivized to have fewer transfers in order to minimize the objective function.

In [30]:
x = mdl.addVars(station_time, vtype=GRB.BINARY, name='x')  # 1 if b_{i,t} > 0 
w = mdl.addVars(station_time, vtype=GRB.BINARY, name='w')  # 1 if b_{i,t} < 0

big_m = 10_000  # large number
# relation between b and x
mdl.addConstrs((b[i, t] <= big_m * x[i, t] for i, t in station_time), 'rel_b_x')
# relation between b and w
mdl.addConstrs((-b[i, t] <= big_m * w[i, t] for i, t in station_time), 'rel_b_w')

# new objective
obj = l.sum() + (x.sum() + w.sum())
mdl.setObjective(obj, GRB.MINIMIZE)
mdl.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3363 rows, 2304 columns and 5568 nonzeros
Model fingerprint: 0x263e55e9
Model has 864 general constraints
Variable types: 1728 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start from previous solve produced solution with objective 215 (0.04s)
MIP start from previous solve produced solution with objective 193 (0.06s)
MIP start from previous solve produced solution with objective 161 (0.09s)
MIP start from previous solve produced solution with objective 151 (0.11s)
MIP start from previous solve produced solution with objective 147 (0.15s)
Loaded MIP start from previous solve with objective 147

Presolve removed 2759 rows and 1642 columns
Presolve time: 0.03s
Presolved: 604 rows, 662 

## Post Processing

In [31]:
df = pd.DataFrame()
if mdl.status == GRB.Status.OPTIMAL:
    df = flow_df.copy()
    df = df.merge(stations[['capacity']], left_on='station', right_index=True)
    df[['bikes_added_or_removed', 'loss_sale', 'beginning_inventory']] = 0
    for k, v in b.items():
        df.loc[k, 'bikes_added_or_removed'] = v.x
        df.loc[k, 'beginning_inventory'] = q[k].x
        df.loc[k, 'loss_sale'] = l[k].x
    df.reset_index(inplace=True)
    print(f'Total Loss : {mdl.objVal}')
    total_bikes_added = df.groupby('time').apply(
        lambda x: x[x.bikes_added_or_removed >= 0]['bikes_added_or_removed'].sum())
    print(f'Total number of bikes added in each hour:\n {total_bikes_added}')
else:
    print('Could not find a feasible solution!')
df.head(10)

Total Loss : 146.99999985848646
Total number of bikes added in each hour:
 time
7    100.0
8    100.0
9    100.0
dtype: float64


,station,time,datetime,start_forecast,end_forecast,date,capacity,bikes_added_or_removed,loss_sale,beginning_inventory
0,1 Ave & E 62 St,7,2022-08-01 07:00:00,11.0,10.0,2022-08-01,45,0.0,1.0,0.0
1,1 Ave & E 62 St,8,2022-08-01 08:00:00,13.0,13.0,2022-08-01,45,0.0,0.0,0.0
2,1 Ave & E 62 St,9,2022-08-01 09:00:00,16.0,19.0,2022-08-01,45,0.0,0.0,0.0
3,1 Ave & E 68 St,7,2022-08-01 07:00:00,17.0,31.0,2022-08-01,62,0.0,0.0,0.0
4,1 Ave & E 68 St,8,2022-08-01 08:00:00,21.0,40.0,2022-08-01,62,0.0,0.0,14.0
5,1 Ave & E 68 St,9,2022-08-01 09:00:00,27.0,47.0,2022-08-01,62,0.0,0.0,33.0
6,1 Ave & E 78 St,7,2022-08-01 07:00:00,9.0,6.0,2022-08-01,57,0.0,3.0,0.0
7,1 Ave & E 78 St,8,2022-08-01 08:00:00,17.0,4.0,2022-08-01,57,13.0,0.0,0.0
8,1 Ave & E 78 St,9,2022-08-01 09:00:00,17.0,6.0,2022-08-01,57,11.0,0.0,0.0
9,10 Ave & W 14 St,7,2022-08-01 07:00:00,3.0,5.0,2022-08-01,55,0.0,0.0,0.0


# Extra: How this problem is solved in reality?

After knowing how many bikes needed in each station, the bikes need to be physically moved from one station to another. 

During rush hours, the traffic is already heavy. So, bicycle trailers (that can usually hold 5 bicycles) are used to move bikes around.

During lighter hours (mainly at night), the bikes are transfered using trucks.

In either case, some bikes should be removed from stations where there are more in-flow of bikes and should be transfered to stations where there are more out-flow of bikes to balance them out. This problem in itself is another mathemtical optimization problem and a variation of the famous Vehicle Routing Problem (VRP) with pickup and delivery.
To learn more, check out [this webinar](https://www.gurobi.com/resource/how-to-synchronize-complex-routing-operations-synched-vrps-with-gurobi/)